In [ ]:
!pip install neo4j
!pip install python-igraph
!pip install easygui
!pip install cairocffi

In [2]:
from neo4j import GraphDatabase
import igraph as ig

In [3]:
driver=GraphDatabase.driver("bolt://3.72.36.113:7687",
                            auth=("neo4j", "ALSSNExercise2"),
                            encrypted = False)
session = driver.session()

In [ ]:
results = session.run("MATCH (n)-[r]-() RETURN count(distinct n) + count(distinct r) as cnt")

for result in results:
    print(result["cnt"])
    print(result)

In [5]:
edges = list()
edge_w = list()
vertices = set()

results = session.run("match (n:Stop)<-[:PART_OF]-(:Stop)-[:LOCATED_AT]-(s:Stoptime)-[:PART_OF_TRIP]-(t:Trip {}) "\
  "-[:PART_OF_TRIP]-(:Stoptime)-[:LOCATED_AT]-(:Stop)-[:PART_OF]->(m:Stop) "\
  "where n.id<>m.id and t.id starts with 'IC'"\
  "return n,m, count(s) as cnt")
g = ig.Graph()

for result in results:
  vertices.add(str(result["n"]["name"]))
  vertices.add(str(result["m"]["name"]))
  edges.append([str(result["n"]["name"]), str(result["m"]["name"])])
  edge_w.append(1/result["cnt"])

g.add_vertices(list(vertices))
g.add_edges(edges)
g.es["weight"]=edge_w

In [ ]:
print(g.is_weighted())
print(g.ecount())

In [ ]:

# Calculate layout
layout_fr = g.layout("fr")

#Define style from network plotting
visual_style = {}
visual_style["vertex_size"] = 1
visual_style["vertex_label_size"]=5
visual_style["vertex_color"] = "red"
visual_style["vertex_label"] = g.vs["name"]
visual_style["layout"] = layout_fr
visual_style["bbox"] = (1200, 800)
visual_style["margin"] = 10

ig.plot(g, **visual_style)